In [12]:
import xarray as xr
from calculate_exposure_to_haz import *

from calculate_exposure_to_haz import INPUT_DIR, EXPOSURES_DIR, get_exposures, write_impact
import numpy as np
exposures_dict = get_exposures(haz_type='FI')


### fire

In [13]:
import pandas as pd
import numpy as np
import xarray as xr

from climada.hazard import Hazard

yearly_hazards = {}

# Loop over each unique year
for year in np.arange(2003,2022):
    file = INPUT_DIR / f"fire_pm25/remapped_monthly/globPMfire_monthly_max_3day_mean_{year}.nc"
    fire = xr.open_dataset(file)
    fire['lon'] = fire['lon'].where(fire['lon'] <= 180, fire['lon'] - 360) # make sure we have the -180 to 180 range

    fire = fire.rename({'lat':'latitude', 'lon':'longitude'})
    fire = fire.drop_vars(['time_bnds'])

    # Convert it into a hazard
    yearly_hazard = Hazard.from_xarray_raster(fire, hazard_type='FI', intensity_unit='kg/m3', intensity='cnc_PM_FRP_m_17')
    yearly_hazard.intensity.data[yearly_hazard.intensity.data<3e-8] = 0
    yearly_hazard.intensity.eliminate_zeros()
    # Append the hazard to the list
    yearly_hazards[year] = yearly_hazard



In [19]:
from pathlib import Path
from climada.entity import Exposures
import numpy as np
exposures_dict = {}


for age in ['all' , '0_1', '65_70_75_80']:
    exposures_dict[age] = {}
    for year in np.arange(2003, 2022):
        year = str(year)
        exposures_dict[age][year] = Exposures.from_hdf5(EXPOSURES_DIR / f"{age}_era5_025_compatible_{year}.hdf5")
        exposures_dict[age][year].gdf["impf_FI"] = 1 
        exposures_dict[age][year].gdf['longitude'] = exposures_dict[age][year].gdf['longitude'].apply(lambda x: x - 360 if x > 180 else x)
        exposures_dict[age][year].gdf['value'][exposures_dict[age][year].gdf['value']<0] = 0


In [15]:
from climada.entity.impact_funcs import ImpactFunc, ImpactFuncSet

impf_set = ImpactFuncSet()

impf25 = ImpactFunc()
impf25.id = 1
impf25.haz_type = 'FI'
impf25.name = 'Step function pm2.5'
impf25.intensity_unit = ''
impf25.intensity = np.array([0, 2.9e-8, 3e-8, 1]) #https://www.thelancet.com/journals/lanplh/article/piis2542-5196(21)00200-X/fulltext
impf25.mdd = np.array([0, 0, 1, 1])
impf25.paa = np.ones(impf25.intensity.shape)
impf25.check()
impf_set.append(impf25)

In [20]:
from climada.engine import Impact, ImpactCalc

impact_dict = {age: {year:ImpactCalc(exposures_dict[age][str(year)], impf_set, yearly_hazards[int(year)]).impact(save_mat=True) for year in exposures_dict[age]} for age in exposures_dict}


impact_dict_no_change_vuln_pop = {age: {year:ImpactCalc(exposures_dict[age]['2003'], impf_set, yearly_hazards[int(year)]).impact(save_mat=True) for year in exposures_dict[age]} for age in exposures_dict}

for age in impact_dict_no_change_vuln_pop:
    for year in impact_dict_no_change_vuln_pop[age]:
        #impact_dict[age][year].tot_value=0
        impact_dict_no_change_vuln_pop[age][year].tot_value=0



2024-08-22 11:23:13,811 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:23:13,823 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:23:13,825 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:23:13,825 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:23:13,826 - climada.engine.impact - WARNING - The 

In [ ]:
from climada.engine import Impact, ImpactCalc

impact_dict = {age: {year:ImpactCalc(exposures_dict[age][str(year)], impf_set, yearly_hazards[int(year)]).impact(save_mat=True) for year in exposures_dict[age]} for age in exposures_dict}


impact_dict_no_change_vuln_pop = {age: {year:ImpactCalc(exposures_dict[age]['2003'], impf_set, yearly_hazards[int(year)]).impact(save_mat=True) for year in exposures_dict[age]} for age in exposures_dict}

for age in impact_dict_no_change_vuln_pop:
    for year in impact_dict_no_change_vuln_pop[age]:
        impact_dict[age][year].tot_value=0
        impact_dict_no_change_vuln_pop[age][year].tot_value=0



2024-08-22 11:29:01,779 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:29:01,781 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:29:01,781 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:29:01,781 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:29:01,782 - climada.engine.impact - WARNING - The 

In [ ]:
impact_concat = {}
impact_concat_no_change_vuln_pop = {}

impact_concat_no_change_vuln_pop = {}
for age in impact_dict_no_change_vuln_pop:
    impact_concat[age] = Impact.concat(list(impact_dict[age].values()),reset_event_ids=True)

    impact_concat_no_change_vuln_pop[age] = Impact.concat(list(impact_dict_no_change_vuln_pop[age].values()),reset_event_ids=True)

In [ ]:

for age in impact_dict:
    write_impact(impact_concat_no_change_vuln_pop[age], 'FI_ec', age)
    write_impact(impact_concat[age], 'FI', age)